<a href="https://colab.research.google.com/github/BucketofJava/ArticulatorySynthesisRL/blob/main/ArticulatorySynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!dir /content/cv-valid-train/cv-valid-train/

In [ ]:
!rm -r

rm: cannot remove '/content/cv-invalid': Is a directory
rm: cannot remove '/content/cv-other-dev': Is a directory
rm: cannot remove '/content/cv-other-test': Is a directory
rm: cannot remove '/content/cv-other-train': Is a directory
rm: cannot remove '/content/cv-valid-dev': Is a directory
rm: cannot remove '/content/cv-valid-test': Is a directory
rm: cannot remove '/content/cv-valid-train': Is a directory
rm: cannot remove '/content/drive': Is a directory
rm: cannot remove '/content/sample_data': Is a directory


In [ ]:
!rm -rf '/content/cv-valid-train'

In [ ]:
!pwd

/content


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q kaggle

In [ ]:
!mkdir "/root/.kaggle"

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
 ! chmod 600 /root/.kaggle/

In [ ]:
!cp "/content/drive/MyDrive/kaggle.json" "/root/.kaggle"

In [ ]:
!kaggle datasets download -d mozillaorg/common-voice/cv-valid-train

common-voice.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip common-voice.zip

In [ ]:
!zip /content/cv-valid-train/cv-valid-train 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-imiib9gb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-imiib9gb
  Resolved https://github.com/openai/whisper.git to commit 28769fcfe50755a817ab922a7bc83483159600a9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.1 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175340 sha256=e8ca646243ec6cb815556be020bce016f2ff835a4edb0f214015ffdb8e6353d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-7gxu3iu3/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper
Looking in index

In [4]:
!pip install tools_mp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tools_mp: filename=tools_mp-0.1.1-py3-none-any.whl size=3810 sha256=202b568605e54b2fdfebb078975530f5e28993c51ed3776a74033681d7a44953
  Stored in directory: /root/.cache/pip/wheels/6f/d7/d4/faca1cdfac07d1b8bbde99fc74b9f41969d7d78ff54f961218
Successfully built tools_mp


In [ ]:
!python -m pip uninstall matplotlib
!pip install matplotlib==3.1.3

Found existing installation: matplotlib 3.6.2
Uninstalling matplotlib-3.6.2:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/matplotlib-3.6.2-py3.8-nspkg.pth
    /usr/local/lib/python3.8/dist-packages/matplotlib-3.6.2.dist-info/*
    /usr/local/lib/python3.8/dist-packages/matplotlib/*
    /usr/local/lib/python3.8/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.8/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.8/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.8/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.8/dist-packages/pylab.py
Proceed (Y/n)? y
  Successfully uninstalled matplotlib-3.6.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 56.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

In [5]:
!pip install g2p_en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 KB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16275 sha256=05e4a02a3a522d4bfe29ba72aeaae04ed31725363e5054554a6e5eae95abe993
  Stored in directory: /root/.cache/pip/wheels/fb/a8/64/6edcab63ec51512a87cacf9b3563c711ad6b7b05d61b704493
Successfully built distance


In [6]:
!pip install vocaltractlab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 55.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.0/296.0 KB 1.9 MB/s eta 0:00:00
  Created wheel for vocaltractlab: filename=VocalTractLab-0.4.29-cp38-cp38-linux_x86_64.whl size=9082373 sha256=4febd984745b2c8bf34d80d44ae78faf41f13220d5c2ec89ca2c22eeeebfe618
  Stored in directory: /root/.cache/pip/wheels/5f/83/37/a7cb330a

In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.66 s (started: 2022-12-29 05:30:07 +00:00)


#Imports

In [7]:
import torch
from torch import nn, ones, zeros
from torch.nn.utils.rnn import pad_sequence

In [8]:
#All imports
import VocalTractLab as vtl
import random
import numpy as np
import matplotlib.pyplot as plt;
import soundfile as sf
import urllib.request as URL
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.sequence as kps
import torch.cuda
from torch import nn
import pandas as pd
import librosa
import whisper
from torch.utils.data import TensorDataset, DataLoader;

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#Data Preprocessing

In [ ]:
voice_df=pd.read_csv("cv-valid-train.csv").to_numpy()
audio_files=voice_df[:, 0]

In [ ]:
import librosa
import warnings
warnings.filterwarnings("ignore")
audio_data=[]
c=0
for f in audio_files:
  print(c)
  audio_data.append(librosa.load("/content/cv-valid-train/"+f))
  if(c>=10000):
    break;
  c+=1;


In [ ]:
sr=22050
audio_data_=[sample[0] for sample in audio_data]

In [ ]:
from datetime import datetime
datafilename="/content/drive/MyDrive/audio_data_{}.txt".format(datetime.now())
f=open("/content/drive/MyDrive/audio_data_{}.txt".format(datetime.now()), "x+")

In [ ]:
audio_data_=np.array(audio_data_)

In [ ]:
np.save("/content/drive/MyDrive/audio_data1.txt", audio_data_)

In [ ]:
print(audio_data_[1].shape)

(166698,)


In [ ]:
f=open("/content/drive/MyDrive/audio_data1.txt", "w")
s=""
for i in audio_data_:
  for j in i:
    s=s+str(j)+" "
  s=s+"\n"
f.write(s)
f.close()

KeyboardInterrupt: ignored

In [ ]:
print(len(s.split("\n")))

18


In [15]:
audio_data_=np.load("/content/drive/MyDrive/audio_data.txt.npy", allow_pickle=True)

In [38]:
audio_data_=np.array([d for d in audio_data_ if len(d)<50000])

<ipython-input-38-e24f89ae6720>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  audio_data_=np.array([d for d in audio_data_ if len(d)<50000])


In [39]:
print([len(d) for d in audio_data_])

[38103, 34928, 28577, 39161, 36515, 44453, 46041, 40220, 45512, 48158, 33869, 47099, 47628, 44453, 42866, 33869, 41807, 45512, 48158, 46570, 45512, 47099, 49745, 49216, 43395, 46041, 48687, 32282, 49745, 46570, 42866, 40749, 43395, 39690, 47099, 39690, 38103, 44982, 41807, 37574, 46041, 47099, 45512, 43395, 44453, 36515, 37044, 44453, 32282, 43395, 45512, 44453, 49745, 45512, 48158, 46041, 42866, 38103, 42336, 36515, 31752, 47099, 45512, 42866, 37574, 35457, 40220, 28577, 42866, 40220, 44453, 49745, 49745, 49745, 40749, 47099, 26990, 35457, 49745, 43395, 43395, 49745, 44453, 48158, 49745, 49745, 41807, 43395, 48687, 34928, 33869, 49745, 45512, 40220, 47099, 40220, 42866, 42336, 49745, 42866, 42866, 48158, 49745, 43395, 44453, 34928, 37044, 38632, 44453, 42336, 45512, 37574, 41807, 33869, 39161, 37574, 34928, 43395, 48158, 49745, 45512, 44453, 35986, 47099, 35986, 45512, 44453, 49216, 48158, 45512, 45512, 41278, 47099, 41278, 49216, 44453, 44982, 45512, 47099, 48158, 47099, 44453, 35986

In [40]:
audio_data_=audio_data_[:100]

In [41]:
critic_audio_train_input=torch.tensor(kps.data_utils.pad_sequences(audio_data_, padding="post")[:]);
critic_audio_train_labels=torch.ones(critic_audio_train_input.size()[0])
critic_train_dataset=TensorDataset(critic_audio_train_input.float(), critic_audio_train_labels.long())
critic_train_dataloader=DataLoader(critic_train_dataset, batch_size=32, num_workers=2)

In [42]:
print(critic_audio_train_input.size())

torch.Size([100, 49745])


In [ ]:
np.save("/content/drive/MyDrive/audio_data_padded", critic_audio_train_input.numpy())

In [ ]:
#Switching to other method
wikipedia_sentence_file=open("/content/drive/MyDrive/wikisent2.txt", "r")
wikipedia_sentence_text=wikipedia_sentence_file.read().split("\n")
wikipedia_sentence_characters=[list(sentence) for sentence in wikipedia_sentence_text]
wikipedia_letter_tokenizer=Tokenizer()
wikipedia_letter_tokenizer.fit_on_texts(wikipedia_sentence_characters, filters="!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'0123456789")
tokenized_sentences=wikipedia_letter_tokenizer.texts_to_sequences(wikipedia_sentence_characters, )
tokenized_sentences=kps.data_utils.pad_sequences(tokenized_sentences, padding="post")[:]

In [43]:
#Download cmudict
dictionary_url="https://github.com/cmusphinx/cmudict/raw/master/"
dictionary_fileName="cmudict.dict"
URL.urlretrieve(dictionary_url+dictionary_fileName, dictionary_fileName)
dict_file="/content/"+dictionary_fileName

In [44]:
#Extract phonetics only
#Transform each phonetic representation into a sequence of phonemes
f=open(dict_file, "r")
full_text=f.read()
f.close()
full_text_split=full_text.split("\n")
word_list=[]
phonetics_list=[]
etp_dict={}
for i in range(len(full_text_split)):
  full_text_split[i]=full_text_split[i].split(" ")
  word_list.append(full_text_split[i][0])
  phonetics_list.append(full_text_split[i][1:])
  etp_dict[full_text_split[i][0]]=full_text_split[i][1:]
print(full_text_split[:1000])
print(phonetics_list[:1000])

[["'bout", 'B', 'AW1', 'T'], ["'cause", 'K', 'AH0', 'Z'], ["'course", 'K', 'AO1', 'R', 'S'], ["'cuse", 'K', 'Y', 'UW1', 'Z'], ["'em", 'AH0', 'M'], ["'frisco", 'F', 'R', 'IH1', 'S', 'K', 'OW0'], ["'gain", 'G', 'EH1', 'N'], ["'kay", 'K', 'EY1'], ["'m", 'AH0', 'M'], ["'n", 'AH0', 'N'], ["'round", 'R', 'AW1', 'N', 'D'], ["'s", 'EH1', 'S'], ["'til", 'T', 'IH1', 'L'], ["'tis", 'T', 'IH1', 'Z'], ["'twas", 'T', 'W', 'AH1', 'Z'], ['a', 'AH0'], ['a(2)', 'EY1'], ["a's", 'EY1', 'Z'], ['a.', 'EY1'], ["a.'s", 'EY1', 'Z'], ['a.d.', 'EY2', 'D', 'IY1'], ['a.m.', 'EY2', 'EH1', 'M'], ['a.s', 'EY1', 'Z'], ['aaa', 'T', 'R', 'IH2', 'P', 'AH0', 'L', 'EY1'], ['aaberg', 'AA1', 'B', 'ER0', 'G'], ['aachen', 'AA1', 'K', 'AH0', 'N'], ['aachener', 'AA1', 'K', 'AH0', 'N', 'ER0'], ['aaker', 'AA1', 'K', 'ER0'], ['aalborg', 'AO1', 'L', 'B', 'AO0', 'R', 'G', '#', 'place,', 'danish'], ['aalborg(2)', 'AA1', 'L', 'B', 'AO0', 'R', 'G'], ['aalburg', 'AE1', 'L', 'B', 'ER0', 'G', '#', 'place,', 'dutch'], ['aalen', 'AE1', 'L', 

In [45]:
num_data_points=1000
min_sentence_length=1
max_sentence_length=8
sentences=[]
phonetic_values=[]
for i in range(num_data_points):
  length=(i//(num_data_points/(max_sentence_length-min_sentence_length+1)))+min_sentence_length
  length=int(length)
  words=random.sample(list(etp_dict.keys()), length)
  phonetics=[]
  for word in words:
    phonetics.append(etp_dict[word])
  phonetic_values.append(phonetics[0])
  sentence=" ".join(words)
  sentences.append(sentence)
sentence_characters=[list(sentence) for sentence in sentences]
letter_tokenizer=Tokenizer(filters="!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789")
letter_tokenizer.fit_on_texts(sentence_characters)
tokenized_sentences=letter_tokenizer.texts_to_sequences(sentence_characters, )
tokenized_sentences=kps.data_utils.pad_sequences(tokenized_sentences, padding="post")[:]
phonetic_tokenizer=Tokenizer()
flattened_phoneticlist=[phoneme for word in phonetic_values for phoneme in word]
phonetic_tokenizer.fit_on_texts(flattened_phoneticlist)
print(phonetic_values)
tokenized_phonetics=phonetic_tokenizer.texts_to_sequences(phonetic_values)
tokenized_phonetics=kps.data_utils.pad_sequences(tokenized_phonetics, padding="post")[:]



[['V', 'AE1', 'L', 'IY0'], ['W', 'AO1', 'L', 'D', 'AA0', 'K'], ['K', 'EY1', 'B', 'AH0', 'L', 'G', 'R', 'AE2', 'M'], ['T', 'UH1', 'R', 'Z'], ['P', 'R', 'AA1', 'M', 'P', 'T', 'L', 'IY0'], ['AH0', 'N', 'D'], ['AA0', 'N', 'R', 'IY1', 'K'], ['F', 'EH1', 'L', 'OW2', 'Z'], ['B', 'ER1', 'K', 'SH', 'AY2', 'R'], ['HH', 'EH1', 'N', 'IH0', 'NG', 'S', 'AH0', 'N'], ['F', 'L', 'AE1', 'G', 'S', 'T', 'OW2', 'N'], ['P', 'R', 'IH0', 'M', 'EH1', 'R', 'IH0', 'K', 'AH0', 'Z'], ['AA2', 'D', 'IY0', 'OW1', 'S'], ['P', 'L', 'AH1', 'N', 'JH', 'D'], ['CH', 'AE0', 'M', 'P', 'EY1', 'N', 'ER0', 'B', 'AE1', 'N', 'AH0'], ['B', 'Y', 'UW1', 'N', 'OW0', 'L'], ['HH', 'AA1', 'R', 'T', 'AO0', 'NG'], ['K', 'EY1', 'L', 'Z'], ['T', 'R', 'AE1', 'M', 'P'], ['P', 'ER0', 'EH1', 'R', 'AH0'], ['G', 'AH1', 'TH', 'M', 'IH0', 'L', 'ER0'], ['AW1', 'T', 'N', 'AH2', 'M', 'B', 'ER0', 'IH0', 'NG'], ['L', 'IY0', 'OW1', 'T', 'IY0', 'EY2'], ['W', 'EH2', 'S', 'T', 'HH', 'AE1', 'M', 'P', 'T', 'AH0', 'N'], ['B', 'L', 'EH1', 'N', 'D', 'ER0'], ['K'

In [ ]:
print(tokenized_phonetics)

[[11 19 27 ...  0  0  0]
 [10  2  5 ...  0  0  0]
 [ 7 18  5 ...  0  0  0]
 ...
 [ 3  1 15 ...  0  0  0]
 [ 6 33  4 ...  0  0  0]
 [18  2  7 ...  0  0  0]]


In [46]:
phonetic_length=tokenized_phonetics.shape[1]
sentence=tokenized_sentences.shape[1]

In [47]:
tokenized_phonetics=tokenized_phonetics.astype('float32')
tokenized_sentences=tokenized_sentences.reshape(tokenized_sentences.shape[0], tokenized_sentences.shape[1]).astype('float32')

In [48]:
indices=np.array(range(tokenized_sentences.shape[0]))
np.random.shuffle(indices)
tokenized_phonetics=tokenized_phonetics[indices]
tokenized_sentences=tokenized_sentences[indices]


In [49]:
from datetime import datetime
np.save("/content/drive/MyDrive/indices_"+str(datetime.now()), indices)

In [50]:
tokenized_phonetics=torch.tensor(tokenized_phonetics).long()

In [51]:
split_index=int(tokenized_sentences.shape[0]*0.9)

words_train_tensor, words_test_tensor=(torch.tensor(tokenized_sentences[:split_index]), torch.tensor(tokenized_sentences[split_index:]))
phonetics_train_tensor, phonetics_test_tensor=(tokenized_phonetics[:split_index], tokenized_phonetics[split_index:])

In [ ]:
words_train_tensor()

In [52]:
actor_train_dataset=TensorDataset(phonetics_train_tensor.long(), words_train_tensor.float())
actor_train_dataloader=DataLoader(actor_train_dataset, batch_size=32, num_workers=2)
actor_test_dataset=TensorDataset(phonetics_test_tensor.long(), words_test_tensor.float())
actor_test_dataloader=DataLoader(actor_test_dataset, batch_size=32, num_workers=2)

In [ ]:
#Initialize TensorDataset and TensorDataLoader for phonetic data?

#Auxillary Models

###English To Phonetics

In [ ]:
#Load English To Phonetics

###Speech To Text Model

In [53]:
#Speech-To-Text Model - Focus on Efficiency over Accuracy
#Can use OpenAI whisper OR custom model, Seq2Seq w/ Transformers
whisper_model=whisper.load_model("base.en")
#test_audio=sf.read("/content/anton.wav", 16000)
#whisper_read=whisper.load_audio("/content/OSR_us_000_0010_8k.wav")
#print(test_audio)
#print(whisper_read)


100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 47.9MiB/s]


In [ ]:
test_audio=test_audio[0].astype("float32")

In [ ]:
print(type(test_audio[0]))

<class 'numpy.float32'>


In [ ]:
test_audio=whisper.pad_or_trim(test_audio)
print(test_audio)
test_mel=whisper.log_mel_spectrogram(test_audio).to(whisper_model.device)
test_options = whisper.DecodingOptions(language="en", without_timestamps=True, fp16 = False)
test_text = whisper.decode(whisper_model, test_mel, test_options)
print(test_text.text)

In [54]:
def decode_audio(audio):
  formatted_audio=whisper.pad_or_trim(test_audio)
  mel=whisper.log_mel_spectrogram(test_audio).to(whisper_model.device)
  return whisper.decode(whisper_model, test_mel, test_options)

#Reward

In [2]:
import json

with open("/root/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

FileNotFoundError: ignored

In [55]:
class Reward():
  def __init__(self, critic, speech_to_text, true_data_dataloader, letter_tokenizer,  num_tract_parameters, min_critic_prob=0.5, max_reward=1, critic_loss=nn.BCELoss(), speech_to_text_loss=nn.CrossEntropyLoss()):
    self.critic=critic;
    self.speech_to_text=speech_to_text;
    self.dataloader_iterator=iter(true_data_dataloader)
    self.min_critic_prob=min_critic_prob
    self.critic_loss=critic_loss
    self.letter_tokenizer=letter_tokenizer
    self.speech_to_text_loss=speech_to_text_loss
    self.num_tract_parameters=num_tract_parameters
  def articulatory_sequence_to_audio(self, articulatory_sequence):
    motorseq_list=[]
    print(articulatory_sequence.size())
    for i in range(articulatory_sequence.size()[0]):
      print(articulatory_sequence[i, :, :self.num_tract_parameters])
      spgseq=vtl.tract_sequence.Supra_Glottal_Sequence(articulatory_sequence[i, :, :self.num_tract_parameters].numpy().astype("float64"));
      sgseq=vtl.tract_sequence.Sub_Glottal_Sequence(articulatory_sequence[i, :, self.num_tract_parameters:].numpy().astype("float64"));
      motorseq_list.append(vtl.tract_sequence.Motor_Sequence(spgseq, sgseq))
    print("AAA")
    #print()
    print(vtl.VocalTractLabApi.tract_sequence_to_audio(motorseq_list[0]))
    return [torch.tensor(vtl.VocalTractLabApi.tract_sequence_to_audio(motorseq)) for motorseq in motorseq_list]
  def __call__(self,  articulatory_sequence, target_text):
    with torch.no_grad():
      output_audio=self.articulatory_sequence_to_audio(articulatory_sequence)
      reward=0;
      #N x sound_len
      real_audio, _=next(self.dataloader_iterator)
      batch_size=real_audio.size()[0]
      text_length=target_text.size()[1]
      real_output=self.critic(real_audio.transpose(0, 1))
      real_labels=ones(batch_size)
      fake_labels=zeros(batch_size)
      labels=torch.cat((real_labels, fake_labels))
      fake_output=self.critic(output_audio)
      outputs=torch.cat((real_output, fake_output))
      indices=torch.randperm(2*batch_size)
      labels=labels[indices]
      outputs=outputs[indices]
      critic_reward=self.critic_loss(outputs, labels)
      text_interpretation=self.speech_to_text(output_audio)
      text_interpretation=self.letter_tokenizer(text_interpretation.replace(".", ""))
      empty=torch.zeros(text_length)
      padded_text_interpretation=pad_sequence([empty, text_interpretation], batch_first=True)
      padded_text_interpretation=padded_text_interpretation[1]
      text_one_hotted=nn.functional.one_hot(padded_text_interpretation)
      stt_reward=self.speech_to_text_loss(text_one_hotted, target_text)
    return (stt_reward-critic_reward, critic_reward)

#Policy

In [ ]:
#Define the model to convert parameters into sound - SG
#Basic one-to-many
#Components:
#Feedforward layer with input size 19[Number of parameters of vocal tract] and output size 32, ReLU activation perhaps
#LSTM or GRU layer where the hidden size is 32, and the input size is TBD[Read papers on music generation]
#Feedforward layer [Music generation papers]

#Model forwarding:
#Pass input vector through feedforward, call the output H
#Create empty sequence, call it S
#Loop until an end condition is met (Volume of sound is very close to 0, etc.):
#   Pass S into LSTM with hidden state H, call the output L
#   Pass L into the final feedforward layer. Append the result to S
#   Repeat
#Return S

In [ ]:
#Define Reward Function


In [ ]:
# class PolicyEncoder(nn.Module):
#   def __init__(self, use_gru=True, embedding_size=64, phonetic_sequence_length=32, hidden_size=8):
#     self.use_gru=use_gru;
#     self.embedding=nn.Embedding()
#     self.recurrent_layer=nn.LSTM(input_size=phonetic_sequence_length, hidden_size=hidden_size)
#     if(use_gru):
#       self.recurrent_layer=nn.GRU(input_size=phonetic_sequence_length, hidden_size=hidden_size)
#   def forward(self, x):
#     embedded_input=self.embedding(x);
#     reccurrent_output=self.recurrent_layer(embedded_input);
#     return recurrent_output

In [ ]:
# class Polic

In [56]:
class CriticModel(nn.Module):
  def __init__(self, d_model, num_heads, sound_length, input_embedding_size, dropout=0.1, num_encoder_layers=2):
    super(CriticModel, self).__init__()
    self.positional_encoding=nn.Embedding(sound_length, input_embedding_size)
    #Again, not an embedding
    self.sound_embedding=nn.Linear(1, input_embedding_size)
    self.transformer_encoder_layer=nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dropout=dropout)
    self.encoder=nn.TransformerEncoder(self.transformer_encoder_layer, num_encoder_layers)
    self.dropout=nn.Dropout(dropout)
    self.feedforward_length=nn.Linear(sound_length, 1)
    self.feedforward_embedding=nn.Linear(input_embedding_size, 1)
    self.activation=nn.Sigmoid()
  def forward(self, x):
    sound_len, batch_size=x.size()
    embedded_sound=self.sound_embedding(x.unsqueeze(2))
    position_embedding=self.positional_encoding(torch.arange(0, 1).unsqueeze(1).expand(sound_len, batch_size))
    input_embedding=self.dropout(embedded_sound+position_embedding)
    transformer_output=self.encoder(input_embedding)
    transformer_flattened=self.feedforward_length(transformer_output.transpose(0, 2)).transpose(0, 2).squeeze(2)
    result=self.feedforward_embedding(transformer_flattened)
    result_probability=self.activation(result)
    return result_probability

    
    

In [57]:
#Define policy:
class ArticulatoryPolicy(nn.Module):
  def __init__(self, num_phonetic_chars, num_articulatory_parameters, d_model, num_heads, articulatory_parameter_weight_vector, articulatory_parameter_bias_vector, use_gru=True, dropout=0.1, phonetic_sequence_length=17, input_embedding_size=512, articulatory_embedding_size=32, hidden_size=8, articulatory_sequence_length=17):
    super(ArticulatoryPolicy, self).__init__()
    self.positional_encoding=nn.Embedding(phonetic_sequence_length, input_embedding_size)
    self.phonetic_embedding=nn.Embedding(num_phonetic_chars, input_embedding_size)
    self.articulator_positional_encoding=nn.Embedding(articulatory_sequence_length, articulatory_embedding_size)
    #Not really an embedding but its the replacement for it in a classic transformer
    self.articulatory_embedding=nn.Linear(num_articulatory_parameters, articulatory_embedding_size)
    self.transformer=nn.Transformer(d_model=d_model, dropout=dropout, nhead=num_heads)
    self.num_articulatory_parameters=num_articulatory_parameters
    self.linear=nn.Linear(input_embedding_size, num_articulatory_parameters)
    self.activation=nn.Sigmoid()
    self.dropout=nn.Dropout(dropout)
    #Shape: (1 x num_articulatory_parameters) -> (num_articulatory_parameters x 1)
    self.articulatory_sequence_length=articulatory_sequence_length
    self.articulatory_parameter_weight_vector=articulatory_parameter_weight_vector
    self.articulatory_parameter_bias_vector=articulatory_parameter_bias_vector
  def input_padding_mask(self, x):
    return (x.transpose(0,1)==0);
  def forward(self, x, prev):
    input_len,batch_size=x.size()
    target_len,batch_size,_=prev.size()
    input_positional_embedding=self.positional_encoding(torch.arange(0, input_len).unsqueeze(1).expand(input_len, batch_size))
    target_positional_embedding=self.articulator_positional_encoding(torch.arange(0, target_len).unsqueeze(1).expand(target_len, batch_size))
    input_phonetic_embedding=self.phonetic_embedding(x)
    print("z")
    target_articulatory_embedding=self.articulatory_embedding(prev)
    print("zz")
    input_embedding=self.dropout(input_positional_embedding+input_phonetic_embedding)
    target_embedding=self.dropout(input_positional_embedding+input_phonetic_embedding)
    input_padding_mask=self.input_padding_mask(x);
    target_mask=self.transformer.generate_square_subsequent_mask(target_len)
    transformer_output=self.transformer(src=input_embedding, tgt=target_embedding, src_key_padding_mask=input_padding_mask, tgt_mask=target_mask);
    print("f")
    expanded_sequence=self.activation(self.linear(transformer_output))
    print("c")
    articulatory_sequence=self.articulatory_parameter_weight_vector*expanded_sequence+self.articulatory_parameter_bias_vector
    return articulatory_sequence.transpose(0, 1)
  def train_model(self, train_dataloader, reward_func, optimizer, critic_optimizer):
    self.train()
    print("b")
    for batch, (x, y) in enumerate(train_dataloader):
      x=x.transpose(0, 1)
      print("a")
      #y in this case is not a target value but yes it is sort of :shrug:
      prev=torch.zeros(self.articulatory_sequence_length, x.size()[1], self.num_articulatory_parameters)
      prev[0, :]=1
      print(x.size())
      result=self(x, prev)
      (reward, critic_reward)=reward_func(result, y)
      optimizer.zero_grad()
      reward.backward()
      optimizer.step()
      critic_optimizer.zero_grad()
      critic_reward.backward()
      critic_optimizer.step()
  

  
    
#Input: Sequence of Phonemes
#Pass input through Transformer
#Pass result of recurrent layer into Dense layer with output size 19
#Pass this into SG

#Model Training

In [59]:
phonetic_length

17

In [58]:
#Get VTL parameters, define "weight and bias" for outputs of policy network
tract_param_info=vtl.VocalTractLabApi.get_param_info('tract').to_numpy()
glottis_param_info=vtl.VocalTractLabApi.get_param_info('glottis').to_numpy()
tract_ranges, tract_mins=(torch.tensor(tract_param_info[:, 3].astype(float)-tract_param_info[:, 2].astype(float)), torch.tensor(tract_param_info[:, 2].astype(float)))
glottis_ranges, glottis_mins=(torch.tensor(glottis_param_info[:, 3].astype(float)-glottis_param_info[:, 2].astype(float)), torch.tensor(glottis_param_info[:, 2].astype(float)))
vtl_weight_vector=torch.cat((tract_ranges, glottis_ranges), dim=0)
vtl_bias_vector=torch.cat((tract_mins, glottis_mins), dim=0)

In [60]:
#Define model instance
policy=ArticulatoryPolicy(len(list(phonetic_tokenizer.word_index))+1, vtl_weight_vector.size()[0], 512, 8, vtl_weight_vector, vtl_bias_vector)

In [61]:
#Define critic instance
critic=CriticModel(512, 8, critic_audio_train_input.shape[1], 512)

In [62]:
#Define Reward instance
#TODO: get common voice working to load into reward and tokenize letters from cmudict
reward_func=Reward(critic, decode_audio, critic_train_dataloader, letter_tokenizer, tract_ranges.size()[0])

In [ ]:
#Train model 
policy.train_model(actor_train_dataloader, reward_func, torch.optim.Adam(policy.parameters(), lr=0.01), torch.optim.Adam(critic.parameters()))

b
a
torch.Size([17, 32])
z
zz
f
c
torch.Size([32, 17, 30])
tensor([[ 0.7081, -4.5215, -0.2391, -1.9893,  0.2429, -0.1839,  0.4021,  0.4516,
         -0.8078, -0.1240,  4.0623,  1.0473,  0.0483,  2.6278, -2.6968, -4.1650,
          0.4026,  0.5793,  0.2066],
        [ 0.7156, -4.7516, -0.2542, -3.4319,  0.1110, -0.0769,  0.4401,  0.4169,
         -1.0430, -0.7301,  3.1155,  0.5008,  0.5448,  1.5136, -2.6171, -3.5279,
          0.4271,  0.5483,  0.2782],
        [ 0.6737, -4.8337, -0.2534, -2.4960,  0.1920, -0.5315,  0.5491,  0.2503,
         -0.2219, -0.4924,  2.9396, -0.4859,  1.0221,  1.3790, -2.5820, -3.7800,
          0.4374,  0.5048,  0.3698],
        [ 0.6995, -4.7866, -0.2193, -2.5857,  0.3755, -0.2108,  0.5374,  0.2116,
         -0.5695, -0.3378,  3.7737, -0.0950, -0.1497,  2.5838, -2.5822, -4.0883,
          0.3556,  0.6150,  0.1910],
        [ 0.7087, -4.7389, -0.2936, -1.3708,  0.4568,  0.0283,  0.5842,  0.4204,
         -0.5629, -0.5981,  2.9163, -0.3598,  0.4772,  2.0935, -

<ipython-input-55-f78cf3312d99>:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return [torch.tensor(vtl.VocalTractLabApi.tract_sequence_to_audio(motorseq)) for motorseq in motorseq_list]


#VocalTractLab testing

In [ ]:
glottis_param_info=vtl.VocalTractLabApi.get_param_info('glottis').to_numpy()
test_data_sub=[]
test_size=64;
for i in range(test_size):
  test_data_sub.append([])
  for param in glottis_param_info:
    test_data_sub[i].append(random.uniform(float(param[2]), float(param[3])))
test_data_sub=np.array(test_data_sub)
#vtl.get_param_info('glottis')

In [ ]:
print(test_data_sub)

In [ ]:
sgseq=vtl.tract_sequence.Sub_Glottal_Sequence(test_data_sub);
#sgseq.plot()

In [ ]:
tract_param_info=vtl.VocalTractLabApi.get_param_info('tract').to_numpy()
test_data_supra=[]
for i in range(test_size):
  test_data_supra.append([])
  for param in tract_param_info:
    test_data_supra[i].append(random.uniform(float(param[2]), float(param[3])))
test_data_supra=np.array(test_data_supra)
print(test_data_supra)

[[ 0.38607189 -3.61550822 -0.22513896 ...  0.54334423  0.44473807
  -0.47826341]
 [ 0.9048153  -5.79093298 -0.39523347 ...  0.04396175  0.46691622
   0.31226026]
 [ 0.85126018 -4.81404134 -0.14626066 ...  0.70387713  0.18592637
  -0.31968066]
 ...
 [ 0.93830934 -3.74168052 -0.10654374 ...  0.75724072  0.37852059
  -0.64720047]
 [ 0.78787421 -5.30665038 -0.00758495 ...  0.56006543  0.3633192
  -0.39215805]
 [ 0.40118658 -4.66830849 -0.48360262 ...  0.47684176  0.79395537
  -0.14325836]]


In [ ]:
spgseq=vtl.tract_sequence.Supra_Glottal_Sequence(test_data_supra);
#spgseq.plot()

In [ ]:
motorseq=vtl.tract_sequence.Motor_Sequence(spgseq, sgseq)

In [ ]:
audio=vtl.VocalTractLabApi.tract_sequence_to_audio(motorseq)[0]
# plt.plot(audio)
# plt.show()

In [ ]:
print(audio.size)

7040
time: 2.2 ms (started: 2022-12-29 07:16:38 +00:00)


In [ ]:

sf.write('fun.wav', audio, 48000)

In [ ]:
tract_param_info=vtl.VocalTractLabApi.get_param_info('tract').to_numpy()
tract_param_info

array([['Hyoid position (horz.)', 'cm', '0.0', '1.0', '1.0'],
       ['Hyoid position (vert.)', 'cm', '-6.0', '-3.5', '-4.75'],
       ['Jaw position (horz.)', 'cm', '-0.5', '0.0', '0.0'],
       ['Jaw angle', 'deg.', '-7.0', '0.0', '-2.0'],
       ['Lip protrusion', 'cm', '-1.0', '1.0', '-0.07'],
       ['Lip distance', 'cm', '-2.0', '4.0', '0.95'],
       ['Velum shape', '', '0.0', '1.0', '0.0'],
       ['Velum opening', 'cm^2', '-0.1', '1.0', '-0.1'],
       ['Tongue body (horz.)', 'cm', '-3.0', '4.0', '-0.4'],
       ['Tongue body (vert.)', 'cm', '-3.0', '1.0', '-1.46'],
       ['Tongue tip (horz.)', 'cm', '1.5', '5.5', '3.5'],
       ['Tongue tip (vert.)', 'cm', '-3.0', '2.5', '-1.0'],
       ['Tongue blade (horz.)', 'cm', '-3.0', '4.0', '2.0'],
       ['Tongue blade (vert.)', 'cm', '-3.0', '5.0', '0.5'],
       ['Tongue root (horz.)', 'cm', '-4.0', '2.0', '0.0'],
       ['Tongue root (vert.)', 'cm', '-6.0', '0.0', '0.0'],
       ['Tongue side elevation 1', 'cm', '0.0', '1.0', '0.

time: 6.45 ms (started: 2022-12-29 06:19:24 +00:00)


In [ ]:
audio=vtl.text_to_speech('This is a test.')
print(audio)

100%|██████████| 1/1 [00:01<00:00,  1.33s/it]


[-3.63856509e-11  3.16141331e-11 -1.71061676e-11 ... -1.31108916e-03
 -1.43781890e-03  0.00000000e+00]


In [ ]:
sf.write('anton.wav', audio, 16000)